In [18]:
import json
import os
import glob
import tensorflow as tf
from object_detection.utils import dataset_util
from PIL import Image
import time
import base64
import numpy as np
import boto3
from concurrent.futures import ThreadPoolExecutor, as_completed

# AWS 인증 정보 설정
aws_access_key_id = 'aws_access_key_id'
aws_secret_access_key = 'aws_secret_access_key'

# S3 버킷 정보
s3_bucket_name = 'tfrecord-bucket01'

# 폴더 경로 설정
root_dir = 'D:/AI_hub'
image_dirs = [os.path.join(root_dir, x, 'jpg') for x in ['Training', 'Validation']]
json_dirs = [os.path.join(root_dir, x, 'json') for x in ['Training', 'Validation']]

# Create a dictionary for mapping category names to numerical ids
# label_map.pbtxt 파일 경로
label_map_path = 'C:/map/label_map.pbtxt'

# label_map.pbtxt 파일에서 카테고리 정보 읽기
category_mapping = {}
with open(label_map_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

category_name = None  # category_name 변수 초기화

for line in lines:
    if "name" in line:
        category_name = line.split(":")[1].strip()[1:-1]
    elif "id" in line and category_name is not None:
        category_id = int(line.split(":")[1].strip())
        category_mapping[category_name] = category_id
        category_name = None  # 변수 초기화

def create_tf_example(image_path, annotation):
    # 이미지 정보 추출
    image = Image.open(image_path)
    encoded_image_data = image.tobytes()
    width, height = image.size
    filename = os.path.basename(image_path).encode()
    image_format = b'jpeg'

    # 바운딩 박스 및 클래스 라벨 정보 추출
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    # 추가된 장애물 클래스 관련 정보
    difficulty = annotation.get('difficulty', 0.0)  # 장애물의 난이도
    occluded = annotation.get('occluded', 0.0)  # 장애물의 가려짐 정도
    truncated = annotation.get('truncated', 0.0)  # 장애물의 트렁크 처리 정도
    sensor_type = annotation.get('sensor_type', b'')  # 사용된 센서 유형
    sensor_position = annotation.get('sensor_position', 0.0)  # 센서의 위치 또는 방향 정보
    center_x = annotation.get('center_x', 0.0)  # 장애물의 중심 좌표 X
    center_y = annotation.get('center_y', 0.0)  # 장애물의 중심 좌표 Y
    encoded_rgb = annotation.get('encoded_rgb', b'')
    encoded_depth = annotation.get('encoded_depth', b'')

    # Check if 'annotations' key exists in the JSON data
    if 'annotations' in annotation:
        for ann in annotation['annotations']:
            segmentation = ann.get('segmentation', None)

            # Check if segmentation points exist
            if segmentation:
                xmin = min(point[0] for point in segmentation) / width
                xmax = max(point[0] for point in segmentation) / width
                ymin = min(point[1] for point in segmentation) / height
                ymax = max(point[1] for point in segmentation) / height

                xmins.append(xmin)
                xmaxs.append(xmax)
                ymins.append(ymin)
                ymaxs.append(ymax)
                classes_text.append(ann['category_id'].encode())
                classes.append(category_mapping.get(ann['category_id'], 0))

    # 인코딩된 RGB 이미지 데이터를 base64로 디코딩
    decoded_rgb = base64.b64decode(encoded_rgb)
    # 디코딩된 데이터를 BytesList 형식으로 변환
    encoded_rgb_feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[decoded_rgb]))

    # 추가할 카메라 정보
    camera_position_x = 0.0  # 카메라의 X 좌표
    camera_position_y = 0.0  # 카메라의 Y 좌표
    camera_position_z = 0.0  # 카메라의 Z 좌표
    camera_orientation_x = 0.0  # 카메라의 X 방향
    camera_orientation_y = 0.0  # 카메라의 Y 방향
    camera_orientation_z = 0.0  # 카메라의 Z 방향
    camera_resolution_width = 0  # 카메라의 해상도 가로 크기
    camera_resolution_height = 0  # 카메라의 해상도 세로 크기

    # Create a tf example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        # 추가된 장애물 클래스 관련 정보
        'image/object/difficulty': dataset_util.float_feature(difficulty),
        'image/object/occluded': dataset_util.float_feature(occluded),
        'image/object/truncated': dataset_util.float_feature(truncated),
        'image/sensor_type': dataset_util.bytes_feature(sensor_type),
        'image/sensor_position': dataset_util.float_feature(sensor_position),
        'image/object/center_x': dataset_util.float_feature(center_x),
        'image/object/center_y': dataset_util.float_feature(center_y),
        'image/encoded_rgb': encoded_rgb_feature,
        'image/encoded_depth': dataset_util.bytes_feature(encoded_depth),
        # 카메라 정보
        'image/camera/position_x': dataset_util.float_feature(camera_position_x),
        'image/camera/position_y': dataset_util.float_feature(camera_position_y),
        'image/camera/position_z': dataset_util.float_feature(camera_position_z),
        'image/camera/orientation_x': dataset_util.float_feature(camera_orientation_x),
        'image/camera/orientation_y': dataset_util.float_feature(camera_orientation_y),
        'image/camera/orientation_z': dataset_util.float_feature(camera_orientation_z),
        'image/camera/resolution_width': dataset_util.int64_feature(camera_resolution_width),
        'image/camera/resolution_height': dataset_util.int64_feature(camera_resolution_height),
    }))

    return tf_example

# Boto3를 사용하여 S3에 파일 업로드
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# 처리할 파일의 총 개수 계산
total_files = sum(len(glob.glob(os.path.join(json_dir, '*.json'))) for json_dir in json_dirs)

# 동시에 처리할 파일의 최대 개수
MAX_WORKERS = 10

def process_file(json_file, image_dir):
    # json 파일 불러오기
    try:
        with open(json_file, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
    except Exception as e:
        print(f"파일 열기 실패: {json_file}, 오류 메시지: {str(e)}")
        return

    # 이미지 파일 경로
    image_filename = os.path.splitext(os.path.basename(json_file))[0].replace('_s', '') + '.jpg'
    image_path = os.path.join(image_dir, image_filename)

    # 파일 경로 정규화
    image_path = os.path.abspath(image_path)

    if not os.path.exists(image_path):
        print(f"이미지 파일 경로 오류: {image_path}, 파일이 존재하지 않습니다.")
        return

    # TFRecord 파일 경로
    tfrecord_filename = os.path.splitext(os.path.basename(json_file))[0] + '.tfrecord'
    tfrecord_file = os.path.join(s3_bucket_name, tfrecord_filename)

    # TFRecord 파일을 로컬에 생성
    local_tfrecord_file = os.path.join('D:/tmp', tfrecord_filename)  # 임시 디렉토리를 사용. 필요시 변경 가능

    try:
        with tf.io.TFRecordWriter(local_tfrecord_file) as writer: 
            tf_example = create_tf_example(image_path, data)
            writer.write(tf_example.SerializeToString())
    except Exception as e:
        print(f"TFRecord 파일 생성 실패: {tfrecord_filename}, 오류 메시지: {str(e)}")
        return

    # 로컬에 생성된 TFRecord 파일을 S3에 업로드
    try:
        s3_client.upload_file(local_tfrecord_file, s3_bucket_name, tfrecord_filename, ExtraArgs={'ServerSideEncryption': 'aws:kms'})

        # S3 업로드 완료 후 임시 파일 삭제
        os.remove(local_tfrecord_file)
    except Exception as e:
        print(f"TFRecord 파일 업로드 실패: {tfrecord_filename}, 오류 메시지: {str(e)}")
        return

start_time = time.time()
processed_files = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = []
    
    for image_dir, json_dir in zip(image_dirs, json_dirs):
        json_files = glob.glob(os.path.join(json_dir, '*.json'))
        
        for json_file in json_files:
            futures.append(executor.submit(process_file, json_file, image_dir))

    for future in as_completed(futures):
        # 예외 처리
        if future.exception() is not None:
            print(f"예외 발생: {future.exception()}")
        else:
            # 현재 처리 중인 파일 및 예상 완료 시간 출력
            processed_files += 1
            elapsed_time = time.time() - start_time
            average_time_per_file = elapsed_time / processed_files if processed_files > 0 else 0
            remaining_files = total_files - processed_files
            remaining_time = average_time_per_file * remaining_files
            
            if processed_files % 20 == 0 :
                print(f"현재 처리 중인 파일: {processed_files}/{total_files}, 예상 완료 시간: {remaining_time:.2f} 초")

print("전체 처리가 완료되었습니다.")

현재 처리 중인 파일: 20/270000, 예상 완료 시간: 321293.05 초
현재 처리 중인 파일: 40/270000, 예상 완료 시간: 267093.28 초
현재 처리 중인 파일: 60/270000, 예상 완료 시간: 239866.26 초
현재 처리 중인 파일: 80/270000, 예상 완료 시간: 225584.09 초
현재 처리 중인 파일: 100/270000, 예상 완료 시간: 221744.26 초
현재 처리 중인 파일: 120/270000, 예상 완료 시간: 211486.79 초
현재 처리 중인 파일: 140/270000, 예상 완료 시간: 206367.99 초
현재 처리 중인 파일: 160/270000, 예상 완료 시간: 203617.82 초
현재 처리 중인 파일: 180/270000, 예상 완료 시간: 200515.36 초
현재 처리 중인 파일: 200/270000, 예상 완료 시간: 198428.76 초
현재 처리 중인 파일: 220/270000, 예상 완료 시간: 197409.44 초
현재 처리 중인 파일: 240/270000, 예상 완료 시간: 195677.17 초
현재 처리 중인 파일: 260/270000, 예상 완료 시간: 194148.65 초
현재 처리 중인 파일: 280/270000, 예상 완료 시간: 194781.32 초
현재 처리 중인 파일: 300/270000, 예상 완료 시간: 193852.42 초
현재 처리 중인 파일: 320/270000, 예상 완료 시간: 192885.66 초
현재 처리 중인 파일: 340/270000, 예상 완료 시간: 192707.43 초
현재 처리 중인 파일: 360/270000, 예상 완료 시간: 192386.45 초
현재 처리 중인 파일: 380/270000, 예상 완료 시간: 192230.36 초
현재 처리 중인 파일: 400/270000, 예상 완료 시간: 191790.88 초
현재 처리 중인 파일: 420/270000, 예상 완료 시간: 192876.26 초
현재 처리 중인 파일: 440/

현재 처리 중인 파일: 3460/270000, 예상 완료 시간: 181139.32 초
현재 처리 중인 파일: 3480/270000, 예상 완료 시간: 181246.11 초
현재 처리 중인 파일: 3500/270000, 예상 완료 시간: 181172.60 초
현재 처리 중인 파일: 3520/270000, 예상 완료 시간: 181186.17 초
현재 처리 중인 파일: 3540/270000, 예상 완료 시간: 181112.91 초
현재 처리 중인 파일: 3560/270000, 예상 완료 시간: 181010.70 초
현재 처리 중인 파일: 3580/270000, 예상 완료 시간: 181037.97 초
현재 처리 중인 파일: 3600/270000, 예상 완료 시간: 181169.90 초
현재 처리 중인 파일: 3620/270000, 예상 완료 시간: 181154.01 초
현재 처리 중인 파일: 3640/270000, 예상 완료 시간: 181213.60 초
현재 처리 중인 파일: 3660/270000, 예상 완료 시간: 181395.47 초
현재 처리 중인 파일: 3680/270000, 예상 완료 시간: 181471.95 초
현재 처리 중인 파일: 3700/270000, 예상 완료 시간: 181409.84 초
현재 처리 중인 파일: 3720/270000, 예상 완료 시간: 181612.84 초
현재 처리 중인 파일: 3740/270000, 예상 완료 시간: 181846.45 초
현재 처리 중인 파일: 3760/270000, 예상 완료 시간: 181800.14 초
현재 처리 중인 파일: 3780/270000, 예상 완료 시간: 181741.64 초
현재 처리 중인 파일: 3800/270000, 예상 완료 시간: 181690.03 초
현재 처리 중인 파일: 3820/270000, 예상 완료 시간: 181657.02 초
현재 처리 중인 파일: 3840/270000, 예상 완료 시간: 181625.49 초
현재 처리 중인 파일: 3860/270000, 예상 완료 시간: 1816

현재 처리 중인 파일: 6880/270000, 예상 완료 시간: 178811.37 초
현재 처리 중인 파일: 6900/270000, 예상 완료 시간: 178780.46 초
현재 처리 중인 파일: 6920/270000, 예상 완료 시간: 178772.63 초
현재 처리 중인 파일: 6940/270000, 예상 완료 시간: 178713.67 초
현재 처리 중인 파일: 6960/270000, 예상 완료 시간: 178671.07 초
현재 처리 중인 파일: 6980/270000, 예상 완료 시간: 178649.50 초
현재 처리 중인 파일: 7000/270000, 예상 완료 시간: 178625.58 초
현재 처리 중인 파일: 7020/270000, 예상 완료 시간: 178580.91 초
현재 처리 중인 파일: 7040/270000, 예상 완료 시간: 178551.40 초
현재 처리 중인 파일: 7060/270000, 예상 완료 시간: 178503.05 초
현재 처리 중인 파일: 7080/270000, 예상 완료 시간: 178493.11 초
현재 처리 중인 파일: 7100/270000, 예상 완료 시간: 178441.69 초
현재 처리 중인 파일: 7120/270000, 예상 완료 시간: 178393.14 초
현재 처리 중인 파일: 7140/270000, 예상 완료 시간: 178406.30 초
현재 처리 중인 파일: 7160/270000, 예상 완료 시간: 178463.73 초
현재 처리 중인 파일: 7180/270000, 예상 완료 시간: 178405.89 초
현재 처리 중인 파일: 7200/270000, 예상 완료 시간: 178390.79 초
현재 처리 중인 파일: 7220/270000, 예상 완료 시간: 178352.06 초
현재 처리 중인 파일: 7240/270000, 예상 완료 시간: 178292.84 초
현재 처리 중인 파일: 7260/270000, 예상 완료 시간: 178317.93 초
현재 처리 중인 파일: 7280/270000, 예상 완료 시간: 1782

현재 처리 중인 파일: 10300/270000, 예상 완료 시간: 174442.71 초
현재 처리 중인 파일: 10320/270000, 예상 완료 시간: 174407.82 초
현재 처리 중인 파일: 10340/270000, 예상 완료 시간: 174381.08 초
현재 처리 중인 파일: 10360/270000, 예상 완료 시간: 174368.53 초
현재 처리 중인 파일: 10380/270000, 예상 완료 시간: 174406.17 초
현재 처리 중인 파일: 10400/270000, 예상 완료 시간: 174385.40 초
현재 처리 중인 파일: 10420/270000, 예상 완료 시간: 174345.06 초
현재 처리 중인 파일: 10440/270000, 예상 완료 시간: 174310.13 초
현재 처리 중인 파일: 10460/270000, 예상 완료 시간: 174311.78 초
현재 처리 중인 파일: 10480/270000, 예상 완료 시간: 174261.29 초
현재 처리 중인 파일: 10500/270000, 예상 완료 시간: 174251.71 초
현재 처리 중인 파일: 10520/270000, 예상 완료 시간: 174198.39 초
현재 처리 중인 파일: 10540/270000, 예상 완료 시간: 174163.94 초
현재 처리 중인 파일: 10560/270000, 예상 완료 시간: 174125.61 초
현재 처리 중인 파일: 10580/270000, 예상 완료 시간: 174070.93 초
현재 처리 중인 파일: 10600/270000, 예상 완료 시간: 174052.75 초
현재 처리 중인 파일: 10620/270000, 예상 완료 시간: 174017.31 초
현재 처리 중인 파일: 10640/270000, 예상 완료 시간: 174054.90 초
현재 처리 중인 파일: 10660/270000, 예상 완료 시간: 174027.35 초
현재 처리 중인 파일: 10680/270000, 예상 완료 시간: 173998.06 초
현재 처리 중인 파일: 10700/2

현재 처리 중인 파일: 13660/270000, 예상 완료 시간: 175975.34 초
현재 처리 중인 파일: 13680/270000, 예상 완료 시간: 175928.71 초
현재 처리 중인 파일: 13700/270000, 예상 완료 시간: 175902.36 초
현재 처리 중인 파일: 13720/270000, 예상 완료 시간: 175873.80 초
현재 처리 중인 파일: 13740/270000, 예상 완료 시간: 175847.91 초
현재 처리 중인 파일: 13760/270000, 예상 완료 시간: 175845.77 초
현재 처리 중인 파일: 13780/270000, 예상 완료 시간: 175802.74 초
현재 처리 중인 파일: 13800/270000, 예상 완료 시간: 175787.52 초
현재 처리 중인 파일: 13820/270000, 예상 완료 시간: 175768.18 초
현재 처리 중인 파일: 13840/270000, 예상 완료 시간: 175738.48 초
현재 처리 중인 파일: 13860/270000, 예상 완료 시간: 175728.03 초
현재 처리 중인 파일: 13880/270000, 예상 완료 시간: 175715.22 초
현재 처리 중인 파일: 13900/270000, 예상 완료 시간: 175686.01 초
현재 처리 중인 파일: 13920/270000, 예상 완료 시간: 175684.09 초
현재 처리 중인 파일: 13940/270000, 예상 완료 시간: 175666.82 초
현재 처리 중인 파일: 13960/270000, 예상 완료 시간: 175678.78 초
현재 처리 중인 파일: 13980/270000, 예상 완료 시간: 175680.32 초
현재 처리 중인 파일: 14000/270000, 예상 완료 시간: 175642.97 초
현재 처리 중인 파일: 14020/270000, 예상 완료 시간: 175624.54 초
현재 처리 중인 파일: 14040/270000, 예상 완료 시간: 175599.53 초
현재 처리 중인 파일: 14060/2

...

현재 처리 중인 파일: 255020/270000, 예상 완료 시간: 10221.80 초
현재 처리 중인 파일: 255040/270000, 예상 완료 시간: 10208.16 초
현재 처리 중인 파일: 255060/270000, 예상 완료 시간: 10194.49 초
현재 처리 중인 파일: 255080/270000, 예상 완료 시간: 10180.88 초
현재 처리 중인 파일: 255100/270000, 예상 완료 시간: 10167.24 초
현재 처리 중인 파일: 255120/270000, 예상 완료 시간: 10153.53 초
현재 처리 중인 파일: 255140/270000, 예상 완료 시간: 10139.86 초
현재 처리 중인 파일: 255160/270000, 예상 완료 시간: 10126.33 초
현재 처리 중인 파일: 255180/270000, 예상 완료 시간: 10112.63 초
현재 처리 중인 파일: 255200/270000, 예상 완료 시간: 10099.03 초
현재 처리 중인 파일: 255220/270000, 예상 완료 시간: 10085.38 초
현재 처리 중인 파일: 255240/270000, 예상 완료 시간: 10071.78 초
현재 처리 중인 파일: 255260/270000, 예상 완료 시간: 10058.06 초
현재 처리 중인 파일: 255280/270000, 예상 완료 시간: 10044.41 초
현재 처리 중인 파일: 255300/270000, 예상 완료 시간: 10030.71 초
현재 처리 중인 파일: 255320/270000, 예상 완료 시간: 10017.02 초
현재 처리 중인 파일: 255340/270000, 예상 완료 시간: 10003.34 초
현재 처리 중인 파일: 255360/270000, 예상 완료 시간: 9989.71 초
현재 처리 중인 파일: 255380/270000, 예상 완료 시간: 9976.06 초
현재 처리 중인 파일: 255400/270000, 예상 완료 시간: 9962.37 초
현재 처리 중인 파일: 255420/270

현재 처리 중인 파일: 258440/270000, 예상 완료 시간: 7891.68 초
현재 처리 중인 파일: 258460/270000, 예상 완료 시간: 7877.98 초
현재 처리 중인 파일: 258480/270000, 예상 완료 시간: 7864.28 초
현재 처리 중인 파일: 258500/270000, 예상 완료 시간: 7850.63 초
현재 처리 중인 파일: 258520/270000, 예상 완료 시간: 7836.97 초
현재 처리 중인 파일: 258540/270000, 예상 완료 시간: 7823.49 초
현재 처리 중인 파일: 258560/270000, 예상 완료 시간: 7809.89 초
현재 처리 중인 파일: 258580/270000, 예상 완료 시간: 7796.21 초
현재 처리 중인 파일: 258600/270000, 예상 완료 시간: 7782.57 초
현재 처리 중인 파일: 258620/270000, 예상 완료 시간: 7768.89 초
현재 처리 중인 파일: 258640/270000, 예상 완료 시간: 7755.27 초
현재 처리 중인 파일: 258660/270000, 예상 완료 시간: 7741.58 초
현재 처리 중인 파일: 258680/270000, 예상 완료 시간: 7727.95 초
현재 처리 중인 파일: 258700/270000, 예상 완료 시간: 7714.24 초
현재 처리 중인 파일: 258720/270000, 예상 완료 시간: 7700.54 초
현재 처리 중인 파일: 258740/270000, 예상 완료 시간: 7686.86 초
현재 처리 중인 파일: 258760/270000, 예상 완료 시간: 7673.26 초
현재 처리 중인 파일: 258780/270000, 예상 완료 시간: 7659.57 초
현재 처리 중인 파일: 258800/270000, 예상 완료 시간: 7645.92 초
현재 처리 중인 파일: 258820/270000, 예상 완료 시간: 7632.22 초
현재 처리 중인 파일: 258840/270000, 예상 완료 시간: 76

현재 처리 중인 파일: 261860/270000, 예상 완료 시간: 5559.54 초
현재 처리 중인 파일: 261880/270000, 예상 완료 시간: 5545.89 초
현재 처리 중인 파일: 261900/270000, 예상 완료 시간: 5532.20 초
현재 처리 중인 파일: 261920/270000, 예상 완료 시간: 5518.49 초
현재 처리 중인 파일: 261940/270000, 예상 완료 시간: 5504.87 초
현재 처리 중인 파일: 261960/270000, 예상 완료 시간: 5491.21 초
현재 처리 중인 파일: 261980/270000, 예상 완료 시간: 5477.52 초
현재 처리 중인 파일: 262000/270000, 예상 완료 시간: 5463.83 초
현재 처리 중인 파일: 262020/270000, 예상 완료 시간: 5450.17 초
현재 처리 중인 파일: 262040/270000, 예상 완료 시간: 5436.48 초
현재 처리 중인 파일: 262060/270000, 예상 완료 시간: 5422.80 초
현재 처리 중인 파일: 262080/270000, 예상 완료 시간: 5409.16 초
현재 처리 중인 파일: 262100/270000, 예상 완료 시간: 5395.45 초
현재 처리 중인 파일: 262120/270000, 예상 완료 시간: 5381.76 초
현재 처리 중인 파일: 262140/270000, 예상 완료 시간: 5368.06 초
현재 처리 중인 파일: 262160/270000, 예상 완료 시간: 5354.41 초
현재 처리 중인 파일: 262180/270000, 예상 완료 시간: 5340.75 초
현재 처리 중인 파일: 262200/270000, 예상 완료 시간: 5327.12 초
현재 처리 중인 파일: 262220/270000, 예상 완료 시간: 5313.44 초
현재 처리 중인 파일: 262240/270000, 예상 완료 시간: 5299.78 초
현재 처리 중인 파일: 262260/270000, 예상 완료 시간: 52

현재 처리 중인 파일: 265280/270000, 예상 완료 시간: 3223.08 초
현재 처리 중인 파일: 265300/270000, 예상 완료 시간: 3209.44 초
현재 처리 중인 파일: 265320/270000, 예상 완료 시간: 3195.80 초
현재 처리 중인 파일: 265340/270000, 예상 완료 시간: 3182.13 초
현재 처리 중인 파일: 265360/270000, 예상 완료 시간: 3168.45 초
현재 처리 중인 파일: 265380/270000, 예상 완료 시간: 3154.81 초
현재 처리 중인 파일: 265400/270000, 예상 완료 시간: 3141.13 초
현재 처리 중인 파일: 265420/270000, 예상 완료 시간: 3127.47 초
현재 처리 중인 파일: 265440/270000, 예상 완료 시간: 3113.81 초
현재 처리 중인 파일: 265460/270000, 예상 완료 시간: 3100.16 초
현재 처리 중인 파일: 265480/270000, 예상 완료 시간: 3086.49 초
현재 처리 중인 파일: 265500/270000, 예상 완료 시간: 3072.83 초
현재 처리 중인 파일: 265520/270000, 예상 완료 시간: 3059.18 초
현재 처리 중인 파일: 265540/270000, 예상 완료 시간: 3045.51 초
현재 처리 중인 파일: 265560/270000, 예상 완료 시간: 3031.84 초
현재 처리 중인 파일: 265580/270000, 예상 완료 시간: 3018.19 초
현재 처리 중인 파일: 265600/270000, 예상 완료 시간: 3004.51 초
현재 처리 중인 파일: 265620/270000, 예상 완료 시간: 2990.85 초
현재 처리 중인 파일: 265640/270000, 예상 완료 시간: 2977.18 초
현재 처리 중인 파일: 265660/270000, 예상 완료 시간: 2963.52 초
현재 처리 중인 파일: 265680/270000, 예상 완료 시간: 29

현재 처리 중인 파일: 268700/270000, 예상 완료 시간: 887.50 초
현재 처리 중인 파일: 268720/270000, 예상 완료 시간: 873.84 초
현재 처리 중인 파일: 268740/270000, 예상 완료 시간: 860.19 초
현재 처리 중인 파일: 268760/270000, 예상 완료 시간: 846.53 초
현재 처리 중인 파일: 268780/270000, 예상 완료 시간: 832.87 초
현재 처리 중인 파일: 268800/270000, 예상 완료 시간: 819.22 초
현재 처리 중인 파일: 268820/270000, 예상 완료 시간: 805.56 초
현재 처리 중인 파일: 268840/270000, 예상 완료 시간: 791.91 초
현재 처리 중인 파일: 268860/270000, 예상 완료 시간: 778.25 초
현재 처리 중인 파일: 268880/270000, 예상 완료 시간: 764.61 초
현재 처리 중인 파일: 268900/270000, 예상 완료 시간: 750.96 초
현재 처리 중인 파일: 268920/270000, 예상 완료 시간: 737.31 초
현재 처리 중인 파일: 268940/270000, 예상 완료 시간: 723.65 초
현재 처리 중인 파일: 268960/270000, 예상 완료 시간: 710.00 초
현재 처리 중인 파일: 268980/270000, 예상 완료 시간: 696.34 초
현재 처리 중인 파일: 269000/270000, 예상 완료 시간: 682.68 초
현재 처리 중인 파일: 269020/270000, 예상 완료 시간: 669.02 초
현재 처리 중인 파일: 269040/270000, 예상 완료 시간: 655.37 초
현재 처리 중인 파일: 269060/270000, 예상 완료 시간: 641.71 초
현재 처리 중인 파일: 269080/270000, 예상 완료 시간: 628.05 초
현재 처리 중인 파일: 269100/270000, 예상 완료 시간: 614.40 초
현재 처리 중인 파일: 